## **Short Term Voltage Forecasting with Graph Neural Networks**

By Karel Križnar, Anton Križnar, Vid Kališnik, Tomo Testen as part of the Stanford CS224W course project.

This notebook accompanies our Medium post on [Short Term Voltage Forecasting with Graph Neural Networks](https://).
The following code loads our database from Github and Drive and uses A3TGCN and GCONV-LSTM for short term voltage prediction.
At the end, trained models are visualized.
You also get to play with our best models.

**Note**: Make sure to **sequentially run all the cells in each section**, so that the intermediate variables / packages will carry over to the next cell.

**Note**: You might need to use GPU for this Colab.
Please click `Runtime` and then `Change runtime type`. Then set the `hardware accelerator` to **GPU**.

# Installation and setup
Here we install and import some important libraries used for GNN.

In [ ]:
import torch
from IPython.display import clear_output

In [ ]:
#This can take some time(even up to 25min)
!pip install torch-scatter -f https://data.pyg.org/whl/torch-1.13.1+cu116.html
!pip install torch-sparse -f https://data.pyg.org/whl/torch-1.13.1+cu116.html
!pip install torch-geometric
!pip install torch-geometric-temporal
clear_output()

Looking in links: https://data.pyg.org/whl/torch-1.13.1+cu116.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.0/108.0 kB 1.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for torch-scatter: filename=torch_scatter-2.1.2-cp310-cp310-linux_x86_64.whl size=3571339 sha256=d7f29946934b83d9f46a8917daecab666e0adc7498374011f1456fbaf817db33
  Stored in directory: /root/.cache/pip/wheels/92/f1/2b/3b46d54b134259f58c8363568569053248040859b1a145b3ce
Successfully built torch-scatter
Looking in links: https://data.pyg.org/whl/torch-1.13.1+cu116.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.0/210.0 kB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
%cd '/content/'
!git clone https://github.com/Kriznar4/GraphVolt.git
%cd GraphVolt
clear_output()

In [ ]:
!pip install gdown
!pip install plotly
clear_output()

In [ ]:
import pandas as pd
import numpy as np
import os
from torch_geometric_temporal.signal import StaticGraphTemporalSignal
import torch
from torch_geometric.data import Data
import sys
sys.path.append('/content/GraphVolt/src/utils')
from utils import read_and_prepare_data, get_array_of_timestemps
from tqdm import tqdm
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch_geometric import seed_everything
import torch.nn.functional as F
from torch_geometric_temporal.nn.recurrent import A3TGCN, GConvLSTM
import matplotlib.pyplot as plt

## Data loading

The network's data is used with SimpleGraphVoltDatasetLoader_Lazy. On initialization of our data loader object, all the preprocessing happens. Node features are put in a torch tensor sorted by date [oldest, …, newest], from which snapshots can be constructed.

In [ ]:
class SimpleGraphVoltDatasetLoader_Lazy(object):
    """
    Check this https://pytorch-geometric-temporal.readthedocs.io/en/latest/_modules/torch_geometric_temporal/dataset/wikimath.html#WikiMathsDatasetLoader
    for an example of how to implement a dataset loader

    And here are the docs https://pytorch-geometric-temporal.readthedocs.io/en/latest/modules/signal.html
    """
    def __init__(self, trafo_id, num_timesteps_in, num_timesteps_out, colab=False):
        self._trafo_id = trafo_id
        self._num_timesteps_in = num_timesteps_in
        self._num_timesteps_out = num_timesteps_out
        self.colab = colab
        self._read_data()
        self._get_edges_and_edge_weights_and_edge_features()
        self._get_targets_and_features()

    def _read_data(self):
        dataset, self.mean_and_std = read_and_prepare_data(self._trafo_id, colab=self.colab) # save in self.mean_and_std
        self._df_edges = dataset["edges_static_data"]
        self._df_measurments = dataset["measurements"]
        self._periods = len(self._df_measurments["date_time"].unique())
        self._node_counts = len(self._df_measurments["node_id"].unique())

    def _get_edges_and_edge_weights_and_edge_features(self):
        self._edges = self._df_edges[["from_node_id", "to_node_id"]].to_numpy().T
        self._edge_features = self._df_edges.drop(["from_node_id", "to_node_id"], axis=1).to_numpy()
        self.num_edge_features = self._edge_features.shape[1]

    def _get_targets_and_features(self):
        #voltage is the 0th column
        #columns names: ['voltage', 'temperature_2m', 'snow_depth', 'cloud_cover', 'is_day',
        #'shortwave_radiation', 'direct_radiation', 'diffuse_radiation',
        #'direct_normal_irradiance', 'active_power', 'reactive_power', 'year',
        #'month', 'day', 'hour', 'minute']

        # voltage_index = 0
        self.voltage_index = self._df_measurments.drop(columns=["date_time", "node_id"]).columns.get_loc("voltage") #TODO: is this ok

        self._dfs = torch.Tensor(get_array_of_timestemps(self._df_measurments))#klobasa

        self.num_features = self._dfs.shape[1]
        self.num_snapshots = self._periods-self._num_timesteps_in-self._num_timesteps_out+1
        self.snapshot_index = range(self.num_snapshots)

    def get_snapshot(self, snapshot_i):
        """
        Returns a snapshot at index snapshot_i of class Data from
        pytorch geometric.
        """
        #Data(x=[113, 21, 12], edge_index=[2, 114], edge_attr=[114, 5], y=[113, 4])

        #voltage_index = 0

        x = torch.Tensor(self._dfs[:,:,snapshot_i:snapshot_i+self._num_timesteps_in])
        y = torch.Tensor(self._dfs[:, self.voltage_index, snapshot_i+self._num_timesteps_in:snapshot_i+self._num_timesteps_in+self._num_timesteps_out])
        edge_index = torch.LongTensor(self._edges)
        edge_attr = torch.Tensor(self._edge_features)

        snapshot = Data(x=x, edge_index=edge_index, edge_attr=edge_attr, y=y)

        return snapshot

    def temporal_signal_split_lazy(self, loader_data_index, train_ratio):
        """
        Splits loader_data_index to two parts.
        """
        split_index = int(train_ratio * len(loader_data_index))

        train = loader_data_index[0:split_index]
        test = loader_data_index[split_index:]

        return train, test

    def temporal_signal_split_lazy_cut(self,loader_data_index, offset=0, number_of_timestemps=2880):
        """
        Gets the data from 'offset' to 'number_of_timestemps' from the data, and the same time period
        just one year later for testing.
        """

        timestemps_in_year = 365*24*60 // 15

        #if we dont have enough data to test one year in advance
        if offset + timestemps_in_year > len(loader_data_index):
            raise ValueError("Offset is too big")

        train = loader_data_index[offset:offset+number_of_timestemps]
        test = loader_data_index[offset + timestemps_in_year : offset + timestemps_in_year + number_of_timestemps]

        return train, test

Load and split data

In [ ]:
trafo_id = "T1330"
num_timesteps_in = 12
num_timesteps_out = 4
test_ratio_vs_eval_ratio = 0.5

manual_seed = 42
seed_everything(manual_seed)

print("Loading data...")
loader = SimpleGraphVoltDatasetLoader_Lazy(trafo_id, num_timesteps_in, num_timesteps_out, colab=True)
print(" done")
loader_data_index = loader.snapshot_index

train_dataset, test_eval_dataset = loader.temporal_signal_split_lazy_cut(loader_data_index)
eval_dataset, test_dataset = loader.temporal_signal_split_lazy(test_eval_dataset, train_ratio=test_ratio_vs_eval_ratio)


Loading data

## Models

In [ ]:
class GNN_A3TGCN(torch.nn.Module):
    def __init__(self, node_features, periods, hidden=32):
        super(GNN_A3TGCN, self).__init__()
        self.name = "GNN_A3TGCN"
        # Attention Temporal Graph Convolutional Cell
        out_channels = hidden
        self.tgnn = A3TGCN(in_channels=node_features,
                           out_channels=out_channels,
                           periods=periods)
        # Equals single-shot prediction
        self.linear = torch.nn.Linear(out_channels, periods)

    def forward(self, x, edge_index):
        """
        x = Node features for T time steps
        edge_index = Graph edge indices
        """
        h = self.tgnn(x, edge_index)
        h = F.relu(h)
        h = self.linear(h)
        return h

class GNN_GCNLSTM(torch.nn.Module):
    def __init__(self,node_features, periods, hidden=32):
        super(GNN_GCNLSTM, self).__init__()
        self.name = "GNN_GCNLSTM"
        out_channels= hidden
        K = 5 # size of Chebyshev filter
        self.recurrent_1 = GConvLSTM(
            in_channels=node_features,
            out_channels=out_channels,
            K=K, normalization='sym',
            bias=False)

        self.linear = torch.nn.Linear(out_channels, periods)

    def forward(self, timesteps, edge_index):
        timesteps = timesteps.permute(2, 0, 1)
        h1, c1 = None, None
        for x in timesteps:
            h1, c1 = self.recurrent_1(x, edge_index, H=h1, C=c1)

        x = F.relu(h1)
        x = self.linear(x)

        return x

# Train_eval

In [ ]:
def train_eval(model, loader, device, train_dataset, eval_dataset, optimizer, loss_fn, scheduler=None, epochs=10, name=""):
    """
    Definition of the training loop.
    """
    epoch_losses_train = []
    epoch_losses_eval = []

    for epoch in range(epochs):
        model.train()
        epoch_loss_train = 0

        for snapshot_i in tqdm(train_dataset, desc="Training epoch {}".format(epoch)):
            snapshot = loader.get_snapshot(snapshot_i)
            snapshot.to(device)
            optimizer.zero_grad()

            out = model(snapshot.x, snapshot.edge_index)
            loss = loss_fn()(out, snapshot.y)
            loss.backward()
            optimizer.step()
            epoch_loss_train += loss.detach().cpu().numpy()

        if scheduler is not None:
            scheduler.step(epoch_loss_train)

        epoch_losses_train.append(epoch_loss_train)

        model.eval()
        epoch_loss_eval = 0
        with torch.no_grad():

            for snapshot_j in tqdm(eval_dataset, desc="Evaluating epoch {}".format(epoch)):
                snapshot = loader.get_snapshot(snapshot_j)
                snapshot.to(device)

                out = model(snapshot.x, snapshot.edge_index)

                loss = loss_fn()(out, snapshot.y).cpu().numpy()
                epoch_loss_eval += loss

            epoch_losses_eval.append(epoch_loss_eval)
            if min(epoch_losses_eval) == epoch_loss_eval:
                torch.save(model.state_dict(), name)
            print("Epoch: {}, Train Loss: {:.7f}, Eval Loss: {:.7f}".format(epoch, epoch_loss_train, epoch_loss_eval))


    return epoch_losses_train, epoch_losses_eval

## Eval

In [ ]:
def eval(model, loader, test_dataset, device, loss_fn, std, mean):
    preds = []
    ys = []
    with torch.no_grad():
        model.eval()
        loss_all = 0
        loss_elementwise = 0

        for snapshot_j in tqdm(test_dataset, desc="Evaluating"):
            snapshot = loader.get_snapshot(snapshot_j)
            snapshot = snapshot.to(device)

            out= model(snapshot.x, snapshot.edge_index)

            loss_all += loss_fn()(out, snapshot.y).cpu().numpy()
            loss_elementwise += loss_fn(reduction="none")(out, snapshot.y).cpu().numpy()

            ys.append(snapshot.y.cpu().numpy()*std+mean)
            preds.append(out.cpu().numpy()*std+mean)

        loss_all *= std/len(test_dataset)
        loss_elementwise *= std/len(test_dataset)

        ys = np.stack(ys, axis=-1)
        preds = np.stack(preds, axis=-1)
    return loss_all, loss_elementwise, preds, ys

## Running the models

**CHOOSE YOUR MODEL HERE**

In [ ]:
#Choose: 'A3TGCN' or 'GCN-LSTM'
model_type = 'GCN-LSTM'

Running the model:

In [ ]:
epochs = 1
device_str = 'cuda'
hidden = 64
learning_rate = 0.01

device = torch.device(device_str)
seed_everything(manual_seed)
if model_type == 'A3TGCN':
  model = GNN_A3TGCN(node_features=loader.num_features, periods=num_timesteps_out, hidden=hidden).to(device)
elif model_type == 'GCN-LSTM':
  model = GNN_GCNLSTM(node_features=loader.num_features, periods=num_timesteps_out, hidden=hidden).to(device)
else:
  print("Wrong model name")

#get dateime string of now
now = pd.Timestamp.now().strftime("%Y%m%d%H%M%S")

name = f"/content/GraphVolt/models/final{model.name}_{now}_{trafo_id}_epochs-{epochs}_in-{num_timesteps_in}_out-{num_timesteps_out}_train-ratio-1month_lr-{learning_rate}_hidden-{hidden}.pt"
name_txt = f"/content/GraphVolt/models/final{model.name}_{now}_{trafo_id}_epochs-{epochs}_in-{num_timesteps_in}_out-{num_timesteps_out}_train-ratio-1month_lr-{learning_rate}_hidden-{hidden}.txt"


optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
loss_fn = torch.nn.MSELoss
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=5, verbose=True)
losses_train, losses_eval = train_eval(model, loader, device, train_dataset, eval_dataset, optimizer, loss_fn, scheduler=scheduler, epochs=epochs, name=name)

In [ ]:
std = loader.mean_and_std["measurements"][1]["voltage"]
mean = loader.mean_and_std["measurements"][0]["voltage"]

In [ ]:
model.load_state_dict(torch.load(name))

In [ ]:
loss_fn = torch.nn.L1Loss

In [ ]:
loss_test, loss_test_elementwise, preds, ys = eval(model, loader, test_dataset, device, loss_fn, std, mean)
loss_test_timewise = loss_test_elementwise.mean(axis=0)
print("Loss all: {:.7f}".format(loss_test))
print("Loss elementwise: {}".format(loss_test_elementwise))

In [ ]:
node = 50
pred_ind = 0

start = 0
len_measurements = 1400

plt.figure(figsize=(20,10))
plt.plot(ys[node,pred_ind,start:start+len_measurements], label="y")
plt.plot(preds[node,pred_ind,start:start+len_measurements], label="pred")
plt.legend()

In [ ]:
#create txt file at name_txt
with open(name_txt, "w") as f:
    #print losses
    f.write("train losses:\n")
    f.write(str(losses_train))
    f.write("\n")
    f.write("eval losses:\n")
    f.write(str(losses_eval))
    f.write("\n")
    f.write("test loss:\n")
    f.write(str(loss_test))
    f.write("\n")
    f.write(str(loss_test_timewise))
    f.write("\n")
    f.write(str(loss_test_elementwise))
    f.write("\n")